<a href="https://colab.research.google.com/github/shivamsouravjha/Toonification/blob/main/generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [447]:
from tensorflow import keras
import cv2
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.models import load_model
import math
import numpy as np
import pathlib
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops
from keras.models import Sequential # to create a cnn model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop,Adam,SGD,Adagrad,Adadelta,Adamax,Nadam
from keras.applications import xception
from keras.layers import LeakyReLU


In [448]:
def boxx(a ,k,n,s):
  x1 = Conv2D(n, (k,k),strides=(s,s), padding='same')(a)
  x1 = LeakyReLU()(x1)
  return x1, x1

In [449]:
def boxy(a,b,k,n):
  x1 = Conv2D(n, (k ,k),strides=(1,1), padding='same')(a)
  x1 = LeakyReLU()(x1)
  x1 = Conv2D(n, (k ,k),strides=(1,1), padding='same')(x1)

  return x1+b,x1+b



In [450]:
input_img = Input(shape=(64,64,1))
inputs= input_img 


In [451]:
a0,b0  = boxx(input_img,7,32,1)  #block 1

In [452]:
a1 = boxx(a0,3,32,2)[0]            #block 2(a)
a2,b2 = boxx(a1,3,64,1)             #block 2(B)

In [453]:
a3 = boxx(a2,3,64,2)[0]            #block 3(a)
a4,b4 = boxx(a3,3,128,1)             #block 3(B)

In [454]:
a5,b5 = boxy(a3,a4,3,128)          #block 4

In [455]:
a6,b6 = boxy(a5,a5,3,128)          #block 5

In [456]:
 a7,b7 = boxy(a6,a6,3,128)          #block 6

In [457]:
 a8,b8 = boxy(a7,a7,3,128)          #block 7

In [458]:
a9 =boxx(a8,3,128,1)[0]            #block 8


In [459]:
a10 = tf.compat.v1.image.resize_bilinear(a9, (tf.shape(a9)[1]*2, tf.shape(a9)[2]*2))

In [460]:
a11 = boxx(a10+a2,3,64,1)[0]
a12 = boxx(a11,3,64,1)[0]
a13 = tf.compat.v1.image.resize_bilinear(a12, (tf.shape(a12)[1]*2, tf.shape(a12)[2]*2))

ValueError: ignored

In [ ]:
autoencoder =Model(input_img,a13)

In [ ]:

from tensorflow.keras.utils import plot_model
plot_model(autoencoder, to_file='mo del.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image("model.png")

In [ ]:
import tensorflow as tf
import numpy as np


def resblock(inputs, out_channel=32, name='resblock'):

        
        x = Conv2D(out_channel, [3, 3], padding='same')(inputs)
        x = tf.nn.leaky_relu(x)
        x = Conv2D(out_channel, [3, 3], padding='same')(x)
        
        return x+inputs
channel = 32
        
x0 = Conv2D(channel, [7, 7],strides=(1,1), padding='same')(inputs)
x0 = tf.nn.leaky_relu(x0)
        

x1 = Conv2D( channel, [3, 3],strides=(2,2), padding='same')(x0)
x1 = tf.nn.leaky_relu(x1)
x1 = Conv2D( channel*2, [3, 3] , padding='same')(x1)
x1 = tf.nn.leaky_relu(x1)
x2 = Conv2D( channel*2, [3, 3],strides=(2,2), padding='same')(x1)
x2 = tf.nn.leaky_relu(x2)
x2 = Conv2D( channel*4, [3, 3], padding='same')(x2)
x2 = tf.nn.leaky_relu(x2)
        
for idx in range(4):
    x2 = resblock(x2, out_channel=channel*4)
x2 = Conv2D( channel*2, [3, 3], padding='same')(x2)
x2 = tf.nn.leaky_relu(x2)
        
h1, w1 = tf.shape(x2)[1], tf.shape(x2)[2]
x3 =  tf.compat.v1.image.resize_bilinear(x2, (h1*2, w1*2))
x3 = Conv2D(channel*2, [3, 3], padding='same')(x3+x1)
x3 = tf.nn.leaky_relu(x3)
x3 = Conv2D( channel, [3, 3], padding='same')(x3)
x3 = tf.nn.leaky_relu(x3)

h2, w2 = tf.shape(x3)[1], tf.shape(x3)[2]
x4 =  tf.compat.v1.image.resize_bilinear(x3, (h2*2, w2*2))
x4 = Conv2D(channel, [3, 3], padding='same')(x4+x0)
x4 = tf.nn.leaky_relu(x4)
x4 =Conv2D( 3, [7, 7], padding='same')(x4)
        
autoencoder =Model(input_img,x4)

In [ ]:

from tensorflow.keras.utils import plot_model
plot_model(autoencoder, to_file='model1.png', show_shapes=True, show_layer_names=True)


In [ ]:

from tensorflow.keras.utils import plot_model
plot_model(autoencoder, to_file='mo del.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image("model.png")